In [1]:
# Pytorch packages
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.utils import to_networkx
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import scatter_add

# Other packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import meshio

In [53]:
# CUDA support
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cpu


In [54]:
class Geometry():
    def __init__(self, mesh_file):

        self.mesh = meshio.read(mesh_file)
        self.node_pos = torch.tensor(self.mesh.points[:, :2], dtype=torch.float, device=device)
        self.mesh_size = self.mesh.points.shape[0]

        self.create_graph()
        self.get_boundary_points()

    def create_graph(self):
        edges = []
        for cell in self.mesh.cells:
            if cell.type == "triangle":
                for element in cell.data:
                    edges.append([element[0], element[1]])
                    edges.append([element[1], element[0]])

                    edges.append([element[1], element[2]])
                    edges.append([element[2], element[1]])

                    edges.append([element[2], element[0]])
                    edges.append([element[0], element[2]])

        edges = torch.tensor(edges, dtype=torch.long, device=device).t().contiguous()
        
        source_nodes = edges[0]
        target_nodes = edges[1]

        source_pos = self.node_pos[source_nodes]
        target_pos = self.node_pos[target_nodes]
        edge_length = torch.norm(source_pos - target_pos, dim=1, keepdim=True)
        
        size = self.node_pos.shape[0]
        f1 = torch.rand(size, device=device).view(-1, 1)
        f2 = torch.rand(size, device=device).view(-1, 1)
        f = torch.cat([f1, f2], dim=1)


        self.graph = Data(x=f1, pos=self.node_pos, edge_index=edges, edge_attr=edge_length).to(device)

        print(self.graph)

    def get_boundary_points(self):
        self.bc_pts = self.mesh.point_sets
        self.keys = self.bc_pts.keys()

In [55]:
mesh_file = "meshes/hole_middle_r10_o1.inp"
obj = Geometry(mesh_file)

Data(x=[5529, 1], edge_index=[2, 64176], edge_attr=[64176, 1], pos=[5529, 2])


In [56]:
data = obj.graph


In [57]:
def comp_grad_fast(data, f, reg=1e-6):
    pos = data.pos  # Node positions
    N, d = pos.shape

    i, j = data.edge_index  # Edge indices

    dx = pos[j] - pos[i]  # Edge direction vectors [E, d]
    du = f[j] - f[i] # Function difference [E, 1]

    weight = (1.0 / (data.edge_attr.view(-1, 1) + 1e-6))  # [E]

    dx_weighted = dx * weight  # [E, d]
    du_weighted = du * weight  # [E, 1]

    outer_dx = dx_weighted.unsqueeze(-1) * dx.unsqueeze(1)  # [E, d, d]
    M = scatter_add(outer_dx, i, dim=0, dim_size=N)  # [N, d, d]

    # Apply weights to `b`
    b = scatter_add(dx_weighted * du_weighted, i, dim=0, dim_size=N)  # [N, d]

    reg_eye = reg * torch.eye(d, device=M.device).unsqueeze(0)  # [1, d, d]
    M_reg = M + reg_eye

    # Invert M for each node in a batch
    M_inv = torch.linalg.inv(M_reg)  # [N, d, d]

    # Compute gradient
    gradients = torch.einsum("nij,nj->ni", M_inv, b)  # [N, d]

    return gradients

In [ ]:
def computegradients(data, f, reg=1e-6):
    pos = data.pos         # [N, d]
    N, d = pos.shape
    i, j = data.edge_index  # i: source nodes, j: target nodes
    dx = pos[j] - pos[i]   # [E, d]
    du = f[j] - f[i]       # [E, 1]

    outer_dx = dx.unsqueeze(-1) * dx.unsqueeze(1)  # [E, d, d]

    M = scatter_add(outer_dx, i, dim=0, dim_size=N)  # [N, d, d]
    b = scatter_add(dx * du, i, dim=0, dim_size=N)     # [N, d]

    reg_eye = reg * torch.eye(d, device=M.device).unsqueeze(0)  # [1, d, d]
    M_reg = M + reg_eye
    M_inv = torch.linalg.inv(M_reg)  # [N, d, d]
    gradients = torch.einsum("nij,nj->ni", M_inv, b)  # [N, d]

    return gradients

In [62]:
f = torch.rand(data.pos.shape[0], device=device).view(-1, 1)
grad = comp_grad_fast(data, f)
print(grad)


tensor([[  188.5773,   -85.3199],
        [  196.3440,  -216.3437],
        [ -118.6271,   -88.8020],
        ...,
        [11309.6865,  1047.0072],
        [  -22.6554,  -186.0572],
        [-5469.2881, -2527.7092]])


In [20]:
t1 = torch.tensor([[1, 2], [2, 4], [3, 6], [4, 8]], dtype=torch.float)
t2 = torch.tensor([[[1]], [[2]], [[3]], [[4]]], dtype=torch.float)
print(t1.shape)
print(t2.shape)

t3 = t1 * t2
print(t3)
print(t3.shape)

torch.Size([4, 2])
torch.Size([4, 1, 1])
tensor([[[ 1.,  2.],
         [ 2.,  4.],
         [ 3.,  6.],
         [ 4.,  8.]],

        [[ 2.,  4.],
         [ 4.,  8.],
         [ 6., 12.],
         [ 8., 16.]],

        [[ 3.,  6.],
         [ 6., 12.],
         [ 9., 18.],
         [12., 24.]],

        [[ 4.,  8.],
         [ 8., 16.],
         [12., 24.],
         [16., 32.]]])
torch.Size([4, 4, 2])


In [ ]:
pos = torch.tensor([
    [0.0, 0.0],  # node 0
    [1.0, 0.0],  # node 1
    [0.0, 1.0],  # node 2
    [1.0, 1.0]   # node 3
])
x_values = pos[:, 0].unsqueeze(1)
print(x_values)
edge_index = torch.tensor([
    [0, 0, 1, 1, 2, 2, 3, 3],
    [1, 2, 0, 3, 0, 3, 1, 2]
])


data = Data(x=x_values, pos=pos, edge_index=edge_index)
print(data)

tensor([[0.],
        [1.],
        [0.],
        [1.]])
Data(x=[4, 2], edge_index=[2, 8])


In [71]:
grad = comp_grad_fast(data, x_values)
print(grad)

tensor([[1.0000, 0.0000],
        [1.0000, 0.0000],
        [1.0000, 0.0000],
        [1.0000, 0.0000]])
